# Guardrailed AI Agent with Security Validators

This notebook implements a self-contained AI agent with built-in guardrails including:
- **Profanity Detection** (using guardrails hub)
- **URL Blocking** (custom validator)
- **Input/Output Validation** (before reaching the LLM)

The agent is protected by safety guardrails that validate both user input and model output.

## 0. Dependencies Installation

Run this cell first to automatically install all required packages and set up guardrails hub validators. This is a one-time setup that ensures all dependencies are available.



**After running this cell, proceed to Cell 1 (API Key Configuration).**

In [47]:
# ============================================================
# NOTEBOOK ENVIRONMENT SETUP — RUN FIRST
# ============================================================

print("🔧 Installing dependencies for Google ADK + Guardrails + Gemini...\n")

# Always upgrade pip inside Jupyter safely
%pip install --upgrade pip

# ------------------------------------------------------------
# Google ADK (agent framework, runner, sessions, callbacks)
# ------------------------------------------------------------
%pip install google-adk

# ------------------------------------------------------------
# Google Gemini Python SDK (genai, types)
# ------------------------------------------------------------
%pip install google-genai

# ------------------------------------------------------------
# Guardrails core (NO API server, NO ML validators)
# ------------------------------------------------------------
%pip install guardrails-ai

# ------------------------------------------------------------
# Jupyter / async / notebook utilities
# ------------------------------------------------------------
%pip install ipywidgets nest-asyncio

print("\n📦 Base Python dependencies installed")

# ------------------------------------------------------------
# Guardrails Hub validator (SAFE, non-ML)
# ------------------------------------------------------------
print("\n🛡️ Installing Guardrails hub validator: profanity_free\n")

!guardrails hub install hub://guardrails/profanity_free

print("\n✅ All dependencies and hub validators installed successfully")
print("🔁 IMPORTANT: Restart the kernel now → Kernel → Restart Kernel → Run All")
print("============================================================")


🔧 Installing dependencies for ADK + Guardrails + Gemini...


  Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-http to determine which version is compatible with other requirements. This could take a while.

  Attempting uninstall: opentelemetry-proto

    Found existing installation: opentelemetry-proto 1.39.1

   ---------------------------------------- 0/6 [opentelemetry-proto]
    Uninstalling opentelemetry-proto-1.39.1:
   ---------------------------------------- 0/6 [opentelemetry-proto]
      Successfully uninstalled opentelemetry-proto-1.39.1
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   --------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-grpc 1.39.1 requires opentelemetry-exporter-otlp-proto-common==1.39.1, but you have opentelemetry-exporter-otlp-proto-common 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.39.1 requires opentelemetry-proto==1.39.1, but you have opentelemetry-proto 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.39.1 requires opentelemetry-sdk~=1.39.1, but you have opentelemetry-sdk 1.37.0 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-http to determine which version is compatible with other requirements. This could take a while.

  Attempting uninstall: opentelemetry-proto

    Found existing installation: opentelemetry-proto 1.37.0

    Uninstalling opentelemetry-proto-1.37.0:

      Successfully uninstalled opentelemetry-proto-1.37.0

   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
   ---------------------------------------- 0/6 [opentelemetry-proto]
  Attempting uninstall: opentelemetry-exporter-otlp-proto-common
   ---------------------------------------- 0/6 [opentelemetry

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-gcp-logging 1.11.0a0 requires opentelemetry-sdk<1.39.0,>=1.35.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.



📦 Base Python packages installed

🛡️ Installing Guardrails hub validator: profanity_free


   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 8.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------------------------------- ------ 1.8/2.2 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.8 MB/s  0:00:00

   ---------------------------------------- 0/3 [widgetsnbextension]
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   --------------------------

## 1. API Key Configuration

Provide your Google API key to enable real LLM calls. Leave blank for simulation mode.

In [3]:
import os

api_key = input("Paste your Google API key: ").strip()

if api_key:
    os.environ["GOOGLE_API_KEY"] = api_key
    print("🟢 LLM MODE: REAL (Google Gemini API enabled)")
else:
    print("🟠 LLM MODE: SIMULATION")


🟢 LLM MODE: REAL (Google Gemini API enabled)


## 2. Core Imports & Warning Suppression

Import all dependencies and suppress guardrails event loop warnings for clean notebook output.

In [4]:
import re
from typing import Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import asyncio

# ============================================================
# SUPPRESS GUARDRAILS EVENT LOOP WARNING
# ============================================================
# Guardrails uses async event loops - suppress the warning
# in Jupyter notebook environments where the event loop
# handling is different. Validation will use synchronous mode.
warnings.filterwarnings(
    "ignore",
    message="Could not obtain an event loop.*",
    category=UserWarning,
    module="guardrails"
)

# Core ADK Imports
from google.adk.agents import Agent as LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest, LlmResponse
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Guardrails imports
from guardrails import Guard
from guardrails.errors import ValidationError
from guardrails.hub import ProfanityFree

print("✅ All imports successful!")
print("✅ Warning filters configured (clean output guaranteed)")
print("✅ ADK Runner and Session imports available for proper agent execution")

✅ All imports successful!
✅ Warning filters configured (clean output guaranteed)
✅ ADK Runner and Session imports available for proper agent execution


## 3. Security Validators

Define custom validators for URL blocking and profanity detection using guardrails hub.

In [5]:
# Create guard for profanity validation
PROFANITY_GUARD = Guard().use(ProfanityFree)

# Custom validator for URL detection
class URLDetector:
    """Custom validator to detect and block URLs in text"""
    
    URL_PATTERN = re.compile(
        r'https?://(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&//=]*)',
        re.IGNORECASE
    )
    
    @staticmethod
    def validate(text: str):
        """Raise ValidationError if URL is found in text"""
        if URLDetector.URL_PATTERN.search(text):
            raise ValidationError("URLs are not allowed in this request")

# Validation functions
def _validate_profanity(text: str):
    """Run Profanity validation"""
    PROFANITY_GUARD.validate(text)

def _validate_url(text: str):
    """Run URL detection validation"""
    URLDetector.validate(text)

def validate_input(text: str):
    """Validate input for profanity and URLs"""
    try:
        _validate_url(text)
        _validate_profanity(text)
    except ValidationError as e:
        raise ValidationError(f"Input validation failed: {str(e)}")

def validate_output(text: str):
    """Validate output for profanity and URLs"""
    try:
        _validate_url(text)
        _validate_profanity(text)
    except ValidationError as e:
        raise ValidationError(f"Output validation failed: {str(e)}")

# Expose objects with a .validate() method
class INPUT_GUARD:
    @staticmethod
    def validate(x): 
        validate_input(x)

class OUTPUT_GUARD:
    @staticmethod
    def validate(x): 
        validate_output(x)

print("✅ Security validators initialized!")

✅ Security validators initialized!


## 4. Guardrail Callbacks

Define input and output guardrail callbacks that intercept and validate LLM requests/responses.

In [6]:
# ============================================================================
# INPUT GUARDRAIL (before_model_callback)
# Blocks unsafe input and prevents the LLM from running.
# ============================================================================
def input_guardrail_callback(
    callback_context: CallbackContext,
    llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Validate the user's latest input before sending to the model."""

    # Get last user message
    user_content = next(
        (c for c in reversed(llm_request.contents) if c.role == "user"),
        None
    )
    if not user_content or not user_content.parts:
        return None

    user_input = user_content.parts[0].text
    print(f"\n--- [INPUT GUARDRAIL CHECK] ---\nChecking input: '{user_input[:60]}...'")

    try:
        # Validate user input (raises ValidationError if unsafe)
        INPUT_GUARD.validate(user_input)
        print("✅ INPUT PASSED. Allowing LLM call.")
        return None  # Allow LLM call

    except ValidationError as e:
        violation_msg = str(e)
        print(f"❌ INPUT BLOCKED. Violations: {violation_msg}")

        rejection_text = (
            f"🚫 GUARDRAIL BLOCKED INPUT 🚫\n"
            f"Your request violates safety policies: {violation_msg}\n"
            f"The model call has been blocked. Please submit a safe request."
        )

        # Return custom response instead of calling the model
        return LlmResponse(
            content=types.Content(
                parts=[types.Part(text=rejection_text)],
                role="model"
            )
        )


# ============================================================================
# OUTPUT GUARDRAIL (after_model_callback)
# Validates model output and replaces unsafe content.
# ============================================================================
def output_guardrail_callback(
    callback_context: CallbackContext,
    llm_response: LlmResponse
) -> Optional[LlmResponse]:
    """Validate model output after generation."""

    response_text = ""
    if llm_response.content and llm_response.content.parts:
        response_text = llm_response.content.parts[0].text

    if not response_text:
        return None

    print(f"\n--- [OUTPUT GUARDRAIL CHECK] ---\nChecking output: '{response_text[:60]}...'")

    try:
        # Validate the model output
        OUTPUT_GUARD.validate(response_text)
        print("✅ OUTPUT PASSED. Sending to user.")
        return None  # Allow output through

    except ValidationError as e:
        violation_msg = str(e)
        print(f"❌ OUTPUT BLOCKED. Violations: {violation_msg}")

        rejection_text = (
            f"🚫 GUARDRAIL BLOCKED OUTPUT 🚫\n"
            f"The generated response violated safety rules: {violation_msg}\n"
            f"The unsafe response has been replaced."
        )

        # Replace unsafe output
        return LlmResponse(
            content=types.Content(
                parts=[types.Part(text=rejection_text)],
                role="model"
            )
        )

print("✅ Guardrail callbacks defined!")

✅ Guardrail callbacks defined!


## 5. Root Agent with Guardrails

Define the LLM agent with attached guardrail callbacks for automatic safety validation.

In [7]:
# ============================================================================
# ROOT AGENT DEFINITION
# ============================================================================
root_agent = LlmAgent(
    name="root_agent",
    model="gemini-2.5-flash",
    description="A general knowledge agent protected by input and output guardrails.",
    instruction=(
        "You are a helpful, ethical, and safe general knowledge assistant. "
        "Always avoid unsafe, harmful, or policy-violating responses."
    ),

    # Guardrail lifecycle hooks
    before_model_callback=input_guardrail_callback,
    after_model_callback=output_guardrail_callback,
)

# Exported agent
agent = root_agent

print("✅ Agent created successfully!")
print(f"Agent Name: {agent.name}")
print(f"Model: {agent.model}")
print(f"Description: {agent.description}")

✅ Agent created successfully!
Agent Name: root_agent
Model: gemini-2.5-flash
Description: A general knowledge agent protected by input and output guardrails.


## 6. Agent Integration Tests

Test the root agent with real scenarios to validate guardrails in action.

In [8]:
print("\n" + "=" * 70)
print("VALIDATOR UNIT TEST SUITE")
print("=" * 70)

test_results = []

# Test 1: Safe input
print("\n[Test 1] Safe input (no URL, no profanity)")
try:
    validate_input('Hello, how can I help you today?')
    print("✅ PASSED: Safe input allowed")
    test_results.append(("Safe Input", True))
except ValidationError as e:
    print(f"❌ FAILED: {e}")
    test_results.append(("Safe Input", False))

# Test 2: Input with HTTPS URL
print("\n[Test 2] Input with HTTPS URL")
try:
    validate_input('Check out this link: https://example.com')
    print("❌ FAILED: Should have blocked URL")
    test_results.append(("HTTPS URL Blocking", False))
except ValidationError as e:
    print(f"✅ PASSED: URL blocked - {e}")
    test_results.append(("HTTPS URL Blocking", True))

# Test 3: Input with HTTP URL
print("\n[Test 3] Input with HTTP URL")
try:
    validate_input('Visit http://google.com for info')
    print("❌ FAILED: Should have blocked URL")
    test_results.append(("HTTP URL Blocking", False))
except ValidationError as e:
    print(f"✅ PASSED: URL blocked - {e}")
    test_results.append(("HTTP URL Blocking", True))

# Test 4: Output validation with safe text
print("\n[Test 4] Output validation (safe)")
try:
    validate_output('This is a safe and helpful response.')
    print("✅ PASSED: Safe output allowed")
    test_results.append(("Safe Output", True))
except ValidationError as e:
    print(f"❌ FAILED: {e}")
    test_results.append(("Safe Output", False))

# Test 5: Output with URL blocking
print("\n[Test 5] Output with URL blocking")
try:
    validate_output('For more information, visit https://website.com')
    print("❌ FAILED: Should have blocked URL in output")
    test_results.append(("Output URL Blocking", False))
except ValidationError as e:
    print(f"✅ PASSED: URL blocked in output - {e}")
    test_results.append(("Output URL Blocking", True))

# Test 6: Complex URL patterns
print("\n[Test 6] URL with path and query parameters")
try:
    validate_input('See https://example.com/path?param=value')
    print("❌ FAILED: Should have blocked complex URL")
    test_results.append(("Complex URL Blocking", False))
except ValidationError as e:
    print(f"✅ PASSED: Complex URL blocked - {e}")
    test_results.append(("Complex URL Blocking", True))

# Test 7: Multiple URLs
print("\n[Test 7] Multiple URLs in input")
try:
    validate_input('Check https://site1.com and https://site2.com')
    print("❌ FAILED: Should have blocked multiple URLs")
    test_results.append(("Multiple URL Blocking", False))
except ValidationError as e:
    print(f"✅ PASSED: Multiple URLs blocked - {e}")
    test_results.append(("Multiple URL Blocking", True))

# Summary
print("\n" + "=" * 70)
print("TEST SUMMARY")
print("=" * 70)

passed = sum(1 for _, result in test_results if result)
total = len(test_results)

for test_name, passed_flag in test_results:
    status = "✅ PASS" if passed_flag else "❌ FAIL"
    print(f"{status} | {test_name}")

print("=" * 70)
print(f"📊 Results: {passed}/{total} tests passed ({100*passed//total}%)")
print("=" * 70 + "\n")


VALIDATOR UNIT TEST SUITE

[Test 1] Safe input (no URL, no profanity)
✅ PASSED: Safe input allowed

[Test 2] Input with HTTPS URL
✅ PASSED: URL blocked - Input validation failed: URLs are not allowed in this request

[Test 3] Input with HTTP URL
✅ PASSED: URL blocked - Input validation failed: URLs are not allowed in this request

[Test 4] Output validation (safe)
✅ PASSED: Safe output allowed

[Test 5] Output with URL blocking
✅ PASSED: URL blocked in output - Output validation failed: URLs are not allowed in this request

[Test 6] URL with path and query parameters
✅ PASSED: Complex URL blocked - Input validation failed: URLs are not allowed in this request

[Test 7] Multiple URLs in input
✅ PASSED: Multiple URLs blocked - Input validation failed: URLs are not allowed in this request

TEST SUMMARY
✅ PASS | Safe Input
✅ PASS | HTTPS URL Blocking
✅ PASS | HTTP URL Blocking
✅ PASS | Safe Output
✅ PASS | Output URL Blocking
✅ PASS | Complex URL Blocking
✅ PASS | Multiple URL Blocking
📊 

## 7. Running Your Tests

Execute all cells in order: Cells 1→7 to run the complete pipeline with dependencies, configuration, and testing.

In [ ]:


print("\n" + "=" * 80)
print("🚀 MULTIPLE TEST CASES — REAL ROOT AGENT (ADK RUNTIME)")
print("=" * 80)

# ------------------------------------------------------------
# Setup ADK Runtime
# ------------------------------------------------------------
APP_NAME = "root_agent_test_suite"
USER_ID = "test_user"
SESSION_ID = "session_main"

session_service = InMemorySessionService()

runner = Runner(
    app_name=APP_NAME,
    agent=root_agent,
    session_service=session_service
)

# ------------------------------------------------------------
# Create session (ASYNC — REQUIRED)
# ------------------------------------------------------------
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("✅ Session created successfully\n")

# ------------------------------------------------------------
# Test cases
# ------------------------------------------------------------
test_cases = [
    {
        "id": 1,
        "name": "Safe Question",
        "input": "What is the capital of France?",
        "expected": "PASS"
    },
    {
        "id": 2,
        "name": "URL in Input (Should Block)",
        "input": "Check this website https://malicious.com for info",
        "expected": "BLOCK_INPUT"
    },
    {
        "id": 3,
        "name": "Normal Knowledge Question",
        "input": "Explain machine learning in simple terms",
        "expected": "PASS"
    },
    {
        "id": 4,
        "name": "Profanity in Output (Should Block)",
        "input": "don't be stupid",
        "expected": "BLOCK_OUTPUT"
    }
]

results = []

# ------------------------------------------------------------
# Run tests
# ------------------------------------------------------------
for test in test_cases:
    print(f"\n{'─' * 80}")
    print(f"[Test {test['id']}] {test['name']}")
    print(f"Expected: {test['expected']}")
    print(f"{'─' * 80}")
    print("📥 User Input:")
    print(test["input"])

    content = types.Content(
        role="user",
        parts=[types.Part(text=test["input"])]
    )

    print("\n→ Running through root_agent...\n")

    final_text = ""

    try:
        for event in runner.run(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        ):
            if hasattr(event, "content") and event.content:
                if hasattr(event.content, "parts"):
                    for part in event.content.parts:
                        if hasattr(part, "text") and part.text:
                            final_text += part.text

        print("📤 Final Response:")
        print("-" * 60)
        print(final_text if final_text else "[NO OUTPUT]")
        print("-" * 60)

        # Basic result classification
        if "BLOCK" in final_text.upper():
            result = "BLOCKED"
        else:
            result = "PASSED"

        results.append((test["name"], result))

    except Exception as e:
        print("🚫 ERROR during execution:")
        print(str(e))
        results.append((test["name"], "ERROR"))

# ------------------------------------------------------------
# Summary
# ------------------------------------------------------------
print("\n" + "=" * 80)
print("📊 FINAL TEST SUMMARY")
print("=" * 80)

for name, result in results:
    if result == "PASSED":
        print(f"✅ {name}: PASSED")
    elif result == "BLOCKED":
        print(f"🚫 {name}: BLOCKED by guardrails")
    else:
        print(f"⚠️  {name}: ERROR")

print("\n✅ CONFIRMED:")
print("   ✓ root_agent executed via ADK Runtime")
print("   ✓ before_model_callback validated input")
print("   ✓ after_model_callback validated output")
print("   ✓ Sessions handled correctly")
print("\n" + "=" * 80 + "\n")




🚀 MULTIPLE TEST CASES — REAL ROOT AGENT (ADK RUNTIME)
✅ Session created successfully


────────────────────────────────────────────────────────────────────────────────
[Test 1] Safe Question
Expected: PASS
────────────────────────────────────────────────────────────────────────────────
📥 User Input:
What is the capital of France?

→ Running through root_agent...


--- [INPUT GUARDRAIL CHECK] ---
Checking input: 'What is the capital of France?...'
✅ INPUT PASSED. Allowing LLM call.

--- [OUTPUT GUARDRAIL CHECK] ---
Checking output: 'The capital of France is Paris....'
✅ OUTPUT PASSED. Sending to user.
📤 Final Response:
------------------------------------------------------------
The capital of France is Paris.
------------------------------------------------------------

────────────────────────────────────────────────────────────────────────────────
[Test 2] URL in Input (Should Block)
Expected: BLOCK_INPUT
──────────────────────────────────────────────────────────────────────────────